# Data Acquisition Example: Testing GPS Receiver System Noise
_By Dan Kuester_

*This implementation (unfortunately) will not work with current ssmdevices - it is based on deprecated driver code*

The noise test sweep here is on a single GPS receiver DUT, excited by an attenuated GPS simulator system.

In [1]:
import ssmdevices as ssm
import labbench as lb

In [ ]:
# -*- coding: utf-8 -*-

import ssmdevices as ssm
import time,serial,duts
from sbp.client import Handler,Framer
from sbp.msg import Container
import pandas as pd
import numpy as np
 
lb.debug_to_screen(lb.WARNING)

# Add support for the with statement
class SwiftNavigationPiksi_v231_USB(duts.SwiftNavigationPiksi_v231_USB):
    ''' Hack the original code so that it's simpler to use here.
    
        GPS device drivers will need a redo if we're going to really use
        them again :)
    '''    
    baud_rate         = 1000000 # Default setting for UART FTDI, can be custom-set using the Piksi Console program
    data_format       = 'sbp'
    usb_serial_id     = r'USB VID:PID=0403:6014 SER=5'#r'FTDIBUS\VID_0403+PID_6014+6&C2017DF&0&3\0000'
    config_commands   = [("SN-RST\r\n")]
    poll_rate = 0.5
    resource = 'COM5'
    
    def connect (self):
        print 'connect!'
        self._port = serial.Serial(self.resource, self.baud_rate, timeout = 0)
   
    def flatten_dict (self,d):
        ret = {}
        
        for k,v in dict(d).items():
            if isinstance(v,Container):
                ret.update(self.flatten_dict(v))
            else:
                ret[k]=v
        
        return ret
       
    def acquire(self, time_seconds):
        ''' Contains the acquisition loop that runs in a background thread.
        '''
        self.connect()
        self.setup()

        t0 = time.time()
        
        data = []
        # Put Serial data onto the queue until the self._stop event triggers
        try:
            with Handler(Framer(self._port.read,None,verbose = True)) as source:
                for msg,metadata in source:
                    if time.time()-t0 > time_seconds:
                        break
                    thisdict = {}
                    if hasattr(msg, 'obs'):
                        for obj in msg.obs:
                            container = self.flatten_dict(obj)
                            sat = container.pop('sat')
                            thisdict.update(zip(['PRN'+str(sat) + ' ' + k for k in container.keys()],container.values()))
                        if hasattr(msg, 'header'):
                            thisdict.update(self.flatten_dict(msg.header))
                        thisdict.update(self.flatten_dict(metadata))
                        data.append(thisdict)
                    if hasattr(msg,'lat'):
                        mydict = [(k,getattr(msg,k)) for k in dir(msg) if (not k.startswith('_') and not hasattr(getattr(msg,k),'__call__') and k != 'payload')]
                        thisdict.update(self.flatten_dict(dict(mydict)))
                        data.append(thisdict)

        finally:
            self.disconnect()

        df = pd.DataFrame(data)
        if 'time' in df.columns:
            df['time'] = pd.to_datetime(df['time'], utc=True)
        df.index.name = 'sample'
        return df

    def __enter__ (self, *args):
        return self
    
    def __exit__ (self, *args):
        try:
            self.disconnect()
        except:
            pass

#import datetime
def cartesian (x,y):
    ''' generate a cartesian product - pairs of each combination of the vectors
        x and y
    '''
    return np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])

#%%
db_path = 'eiin_con.db'

# "Strong signal" level with no excess noise for soak
soak_c_atten  = 21
soak_e0_atten = 110

soak_time = 3*60        # seconds
acquisition_time = 3*60 # seconds

# Generate the grid of c and e0 attenuation levels
points = cartesian(range(21,45,1),[110]+range(55,31,-1))
# ...and then randomize them per Adam's suggestion
np.random.shuffle(points) 

with ssm.instruments.MiniCircuitsRCDAT('11604210021') as c,\
     ssm.instruments.MiniCircuitsRCDAT('11604210004') as e0,\
     ssm.instruments.SpirentGSS8000('COM3')          as gss,\
     SwiftNavigationPiksi_v231_USB('COM5')           as dut,\
     lb.StatesToSQLite(db_path)                      as db:
         
         # Log changes to state settings (e.g., c.state.attenuation)
         # on these devices
         db.observe([c,e0,gss])

         for i,(c_att, n0_att) in enumerate(points):
             print '{}/{}: atten C {} dB, atten N0 {} dB'.format(i+1, len(points), c_att, n0_att)
             
             # Setup GSS8000
             gss.reset()
             gss.run()
             gss.state.gps_week # to go in the dB
             gss.state.current_scenario # to go in the dB
             gss.state.utc_time # to go in the dB
             print '* GSS8000 ready'
             
             # Setup DUT
             dut.setup_device()
             print '* DUT ready'

             # Soak
             print '* soak start'
             c.state.attenuation = soak_c_atten
             e0.state.attenuation = soak_e0_atten
             time.sleep(soak_time)

             # Acquire
             print '* acq start'
             c.state.attenuation = c_att
             e0.state.attenuation = n0_att
             db.commit() # Timestamp taken here!!!      
             data = pd.DataFrame(dut.acquire(acquisition_time))

             # Store
             db.write(soak_time=soak_time, # write states
                      acquisition_time=acquisition_time, 
                      soak_c_atten = soak_c_atten,
                      soak_e0_atten = soak_e0_atten) 

             db.append(data) # add big honkin data
             db.write() # write each iteration, because there are a lot of them and they are long